In [1]:
# Document Q&A Bot with LangChain on Google Colab
# This notebook implements a document QA system conceptually similar to
# Anthropic's Model Context Protocol but using open-source components

# Install required packages
!pip install -q langchain langchain_community langchain-huggingface
!pip install -q faiss-cpu sentence-transformers
!pip install -q transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read)

In [5]:

import os
import uuid
from typing import List, Dict, Any
from IPython.display import Markdown, display

from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

# For running a model locally on Colab
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

class DocumentQABot:
    def __init__(self, model_name="mistralai/Mistral-7B-Instruct-v0.2"):
        """Initialize the Document QA Bot with local model and LangChain components."""
        print("Initializing Document QA Bot...")
        print(f"Loading model: {model_name}")

        # Load model and tokenizer with optimizations for Colab
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,  # Use fp16 for efficiency
            device_map="auto",          # Automatically use available devices
            load_in_8bit=True           # 8-bit quantization to reduce memory usage
        )

        # Setup text generation pipeline
        self.pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_length=2048,
            temperature=0.7,
            top_p=0.95,
            repetition_penalty=1.15
        )

        # Create LangChain LLM
        self.llm = HuggingFacePipeline(pipeline=self.pipe)

        # Initialize embeddings model
        print("Loading embedding model...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
        )

        # Initialize document store
        self.documents = {}
        self.vectorstore = None
        self.conversation_chain = None
        self.chat_history = []

        # Text splitter for document processing
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )

        # Memory for conversation history
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )

        print("Document QA Bot initialized and ready!")

    def add_document(self, document_content: str, document_name: str = None) -> str:
        """Add a document to the context."""
        doc_id = str(uuid.uuid4())
        doc_name = document_name or f"Document-{doc_id[:8]}"

        # Store document metadata
        self.documents[doc_id] = {
            "id": doc_id,
            "name": doc_name,
            "content": document_content
        }

        print(f"Processing document: {doc_name}")

        # Process the document into chunks
        doc_obj = Document(page_content=document_content, metadata={"source": doc_name, "id": doc_id})
        doc_chunks = self.text_splitter.split_documents([doc_obj])

        # Update or create the vector store
        if self.vectorstore is None:
            self.vectorstore = FAISS.from_documents(doc_chunks, self.embeddings)
        else:
            # Add document to existing vectorstore
            self.vectorstore.add_documents(doc_chunks)

        # Create or update the conversation chain with the new document context
        self._update_conversation_chain()

        print(f"Added document '{doc_name}' (ID: {doc_id}) to the context")
        return doc_id

    def _update_conversation_chain(self):
        """Update the conversation chain with the current vectorstore."""
        if self.vectorstore:
            self.conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.vectorstore.as_retriever(
                    search_type="similarity",
                    search_kwargs={"k": 5}  # Retrieve top 5 chunks
                ),
                memory=self.memory,
                return_source_documents=True,
                verbose=True
            )

    def ask(self, question: str) -> Dict[str, Any]:
        """Ask a question about the documents in context."""
        if not self.conversation_chain:
            return {
                "answer": "No documents have been added to the context yet. Please add at least one document before asking questions.",
                "sources": []
            }

        print(f"Question: {question}")
        print("Retrieving relevant document sections...")

        # Use the conversation chain to get an answer
        result = self.conversation_chain({"question": question})

        # Extract source document references
        sources = []
        if "source_documents" in result:
            for doc in result["source_documents"]:
                sources.append({
                    "name": doc.metadata.get("source", "Unknown"),
                    "id": doc.metadata.get("id", "Unknown"),
                    "excerpt": doc.page_content[:150] + "..." if len(doc.page_content) > 150 else doc.page_content
                })

        print("Generated answer based on document context")

        return {
            "answer": result["answer"],
            "sources": sources
        }

    def list_documents(self) -> List[Dict[str, str]]:
        """List all documents in the context."""
        return [{"id": doc_id, "name": doc["name"]} for doc_id, doc in self.documents.items()]

    def remove_document(self, doc_id: str) -> bool:
        """Remove a document from the context."""
        if doc_id in self.documents:
            doc_name = self.documents[doc_id]["name"]
            del self.documents[doc_id]

            print(f"Removing document '{doc_name}' (ID: {doc_id}) from context")
            print("Rebuilding vector store without this document...")

            # For simplicity, we'll rebuild the vectorstore from scratch
            # A more efficient implementation would directly remove vectors from FAISS
            self.vectorstore = None

            # Re-add all remaining documents
            remaining_docs = []
            for d_id, doc in self.documents.items():
                doc_obj = Document(
                    page_content=doc["content"],
                    metadata={"source": doc["name"], "id": d_id}
                )
                remaining_docs.append(doc_obj)

            if remaining_docs:
                doc_chunks = self.text_splitter.split_documents(remaining_docs)
                self.vectorstore = FAISS.from_documents(doc_chunks, self.embeddings)
                self._update_conversation_chain()
            else:
                self.conversation_chain = None

            print(f"Document '{doc_name}' removed from context")
            return True

        print(f"Document with ID {doc_id} not found")
        return False

    def clear_conversation_history(self):
        """Clear the conversation history."""
        self.memory.clear()
        print("Conversation history cleared")


In [6]:
# Run this cell to initialize the bot
# If you have limited resources, use a smaller model
# bot = DocumentQABot("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")  # Smaller model
bot = DocumentQABot("mistralai/Mistral-7B-Instruct-v0.2")        # Standard model

# Add your own documents here
doc1_id = bot.add_document("""
# Company Overview
Acme Corporation was founded in 2010 and specializes in AI solutions for healthcare.
Our annual revenue reached $50 million in 2023, with a 25% year-over-year growth.

## Products
- MedAssist: AI diagnostic tool
- HealthTracker: Patient monitoring system
- DocFlow: Medical documentation automation

## Leadership
- CEO: Jane Smith
- CTO: John Davis
- CFO: Michael Johnson
""", "Acme Company Overview")

doc2_id = bot.add_document("""
# Q4 2023 Financial Report

Acme Corporation closed Q4 with strong performance:
- Revenue: $15.2 million (30% increase from Q3)
- New customers: 45 hospitals and 120 clinics
- MedAssist adoption up 40%

## Challenges
- Supply chain issues delayed HealthTracker 2.0 release
- Increasing competition in the medical AI space

## 2024 Outlook
Planning IPO in Q3 2024 with estimated valuation of $500M
""", "Q4 Financial Report")

# Ask questions about your documents
result = bot.ask("What products does Acme offer?")
print(f"Answer: {result['answer']}")
print("Sources:")
for source in result['sources']:
    print(f"- {source['name']}: {source['excerpt']}")

# Ask another question
result = bot.ask("When is the company planning to go public and what's the valuation?")
print(f"Answer: {result['answer']}")
print("Sources:")
for source in result['sources']:
    print(f"- {source['name']}: {source['excerpt']}")

# Add your own document and ask a question about it
# my_doc_id = bot.add_document("Your document text here", "Your Document Name")
# result = bot.ask("Your question about the document")

# Remove a document when you no longer need it
# bot.remove_document(doc_id)

# Clear conversation history if needed
# bot.clear_conversation_history()

Initializing Document QA Bot...
Loading model: mistralai/Mistral-7B-Instruct-v0.2


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend